# Forward Rate Agreements Fundamentals

This notebook demonstrates how to access and use the main functionalities of **Forward Rate Agreements** (FRA) within **LSEG Financial Analytics SDK**. 

**You will be able to:**
- Define FRA instruments on standard period
- Configure pricing parameters
- Calculate FRA analytics


## Imports

Import the following necessary modules:

- `lseg_analytics.pricing.instruments.forward_rate_agreement` - for Forward Rate Agreement instruments definitions and analytics

This notebook uses external library **pandas**; please ensure it is installed in your Python environment (e.g. 'pip install pandas') before running the code.

In [1]:
from lseg_analytics.pricing.instruments import forward_rate_agreement as fra
import datetime as dt
import json
import pandas as pd

## Data Preparation

To define a FRA instrument you need to follow a structured 3-step process:
1. **FRA Definition** - Specify the FRA instrument definition
2. **FRA Instrument Definition** - Create the instrument object
3. **Pricing Preferences** - Configure pricing parameters, optional

We take here an example of USD LIBOR 2Mx8M FRA priced in historical time (valuation on 2025-07-21)

In [2]:
# Create FRA defintion object
fra_definition = fra.ForwardRateAgreementDefinition(
    start_tenor="2M", # mandatory
    fixed_rate_percent=4,
    end_tenor="8M", # mandatory
    notional_ccy="USD", # mandatory
    index_name="LIBOR",
    notional_amount=1000000
)

# Create FRA instrument defintion object
fra_instrument = fra.ForwardRateAgreementDefinitionInstrument(
    definition = fra_definition
)

# Create FRA pricing parameters object - optional
fra_parameters = fra.ForwardRateAgreementPricingParameters(
    valuation_date  = dt.datetime.strptime("2025-07-21", "%Y-%m-%d"),
)

## Request Execution

In [3]:
# Execute the calculation using the price() function
# The 'definitions' parameter accepts a list of instruments definitions for batch processing

fra_response = fra.price(
    definitions = [fra_instrument], 
    pricing_preferences = fra_parameters
)

## Results Display

#### Key Sections in the `response` JSON

 - **definitions**: Instrument setup (e.g., index name, fixed rate percent, start/end dates/tenors, notional, notional currency), it's ForwardRateAgreementDefinition that we used. 

 - **pricingPreferences**: Valuation date, price side, reporting currency, market data date, tenor reference date.

 - **analytics**:
   - **tabularData**: `data`, `headers`, `statuses`
   - **cashFlows**: gives details on the different cashflows of the FRA product
   - **description**: Instrument summary ForwardRateAgreementDefinition and also the default fields not specified in the ForwardRateAgreementDefinition, but used by default in the calculation
   - **nominalMeasures**: metric on the FRA product: PV01
   - **pricingAnalysis**: different information on market FRA valuation: `marketDataDate`, `valuationDate`, `discountFactor`, `fixedRatePercent`, `parRatePercent`, `spreadBp`
   - **valuation**: `marketValueInDealCcy`, `marketValueInReportCcy`
   - **error**: provides the root cause of the issue, if any

In [4]:
# Access the valuation object
valuation = fra_response.data.analytics[0].valuation
print(json.dumps(valuation.as_dict(), indent=4))

{
    "marketValueInDealCcy": 2336.88876053983
}


In [5]:
# Convert to DataFrame
df_valuation = pd.DataFrame(list(valuation.items()), columns=["Fields", "Value"])
# Display the DataFrame
display(df_valuation)

,Fields,Value
0,marketValueInDealCcy,2336.888761


### Available Classes and Functions in forward_rate_agreement Module

The following is a complete list of all classes, enums, and functions available after importing from `lseg_analytics.pricing.instruments.forward_rate_agreement`:

**Core Classes:**
- `ForwardRateAgreementDefinition` - Define FRA instrument specifications and parameters
- `ForwardRateAgreementDefinitionInstrument` - Instrument container wrapper for pricing operations
- `ForwardRateAgreementPricingParameters` - Configure pricing settings and market data preferences

**Response Classes:**
- `ForwardRateAgreementCalculationResponse` - Main response container
- `ForwardRateAgreementAnalyticsResponseData` - Detailed analytics and calculations
- `ForwardRateAgreementAnalyticsResponseWithError` - Response with error

**Specific FRA Classes within response:**
- `ForwardRateAgreementDescription` - Gives the main characteristics of the instrument
- `ForwardRateAgreementValuation` - Individual instrument valuation results
- `ForwardRateAgreementPricingAnalysis` - Some information on pricing
- `ForwardRateAgreementCashflows` - Detailed FRA cash-flows
- `ForwardRateAgreementNominalMeasures` - Different metrics on FRA instrument

**Configuration Enums:**
- `InterestCalculationMethodEnum` - Day count conventions (DCB_ACTUAL_360, DCB_ACTUAL_365, etc.)
- `PaymentBusinessDayConventionEnum` - Business day conventions (NEXT_BUSINESS_DAY, MODIFIED_FOLLOWING, etc.)
- `PriceSideEnum` - Market price sides (BID, ASK, MID, LAST)
- `TenorReferenceDateEnum` - reference date from which maturity date from tenor is calculated (SPOT_DATE, VALUATION_DATE)

**Main Functions:**
- `price()` - Execute FRA pricing calculations